In [1]:
#Libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [8]:
# machine learing model pipeline
from sklearn.svm import SVR
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,ExpSineSquared, DotProduct,ConstantKernel)
from sklearn.gaussian_process import GaussianProcessRegressor
# Determination of the kernel used and the value lenght_scale at which the optimization process starts
kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e3))
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor


#ML Supporters
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn import preprocessing
import math


In [3]:
#Loading Data
DATA=pd.read_excel(r'ML1-PreMon1.xlsx',sheet_name='Sheet1')
y=pd.Series(DATA.LWP)
X=DATA.loc[:,~DATA.columns.isin(['LWP','Latitude', 'Longitude'])]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [9]:
model_pipeline = []
model_pipeline.append(SVR(C=1.0, epsilon=0.2, kernel='rbf'))
# model_pipeline.append(GaussianProcessRegressor(kernel=kernel, alpha=0.1))
model_pipeline.append(RandomForestRegressor())
model_pipeline.append(DecisionTreeRegressor(max_depth=1))
model_pipeline.append(RANSACRegressor(LinearRegression(),
		max_trials=4, 		# Number of Iterations
		min_samples=2, 		# Minimum size of the sample
		loss='absolute_loss', 	# Metrics for loss
		residual_threshold=10 	# Threshold
		))
model_pipeline.append(GradientBoostingRegressor())


In [10]:

r2_s=[]
RMSE=[]
MAE=[]


for model in model_pipeline:

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2_s.append(r2_score(y_test,y_pred))
    # RMSE.append(math.sqrt(mean_squared_error(y_test, y_pred)))
    # MAE.append(mean_absolute_error(y_test, y_pred))
    


ValueError: All arrays must be of the same length

In [11]:
model_list = ['SVR', 'RandomForestRegressor', 'DecisionTreeRegressor', 'RANSACRegressor','GBRT']
result_df = pd.DataFrame({'Model' : model_list, 'R2_Score': r2_s})
result_df


,Model,R2_Score
0,SVR,0.053816
1,RandomForestRegressor,0.347185
2,DecisionTreeRegressor,0.189600
3,RANSACRegressor,-0.098444
4,GBRT,0.302872


In [25]:
from sklearn.model_selection import GridSearchCV
LR={'n_estimators':[100,150,200,250],'max_depth':[8,12,16,18,20,22,24,26,28,30]}
# ({'learning_rate': 0.05, 'n_estimators': 150}, 0.29390349089746487)#GradientBoosted
# ({'max_depth': 30, 'n_estimators': 250}, 0.33811291142760613)#Random Forest
tuning=GridSearchCV(estimator=RandomForestRegressor(),
            param_grid=LR,scoring='r2')
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_

({'max_depth': 30, 'n_estimators': 250}, 0.33811291142760613)